In [16]:
import pandas as pd
import numpy as np

bom_df = pd.read_excel('../data/2022-05-28_REFUND.xlsx', sheet_name='BOM')
a12_df = pd.read_excel('../data/2022-05-28_REFUND.xlsx', converters={'Số TK': str}, sheet_name='2021-2022.A12', thousands=',')
e62_df = pd.read_excel('../data/2022-05-28_REFUND.xlsx', converters={'Số TK': str}, sheet_name='E62', thousands=',')

def datetime_to_str(row):
    try:
        return row['Ngày ĐK'].strftime('%d/%m/%Y')
    except:
        return row['Ngày ĐK']


a12_df['Ngày ĐK'] = a12_df.apply(lambda row: datetime_to_str(row), axis=1)
a12_df['Ngày ĐK'] = pd.to_datetime(a12_df['Ngày ĐK'], dayfirst=True)

e62_df['Ngày ĐK'] = e62_df.apply(lambda row: datetime_to_str(row), axis=1)
e62_df['Ngày ĐK'] = pd.to_datetime(e62_df['Ngày ĐK'], dayfirst=True)

a12_df = a12_df[['Số TK', 'Ngày ĐK', 'Tổng số lượng', 'Tên hàng', 'Mã NPL/SP', 'Đơn vị tính', 'Tổng trị giá', 'Thuế suất XNK']]
e62_df = e62_df[['Số TK', 'Ngày ĐK', 'Tổng số lượng', 'Tên hàng', 'Mã NPL/SP', 'Đơn vị tính']]

e62_df = e62_df.groupby(['Số TK', 'Ngày ĐK','Tên hàng', 'Mã NPL/SP', 'Đơn vị tính']).aggregate({'Tổng số lượng': 'sum'})
a12_df = a12_df.groupby(['Số TK', 'Ngày ĐK','Tên hàng', 'Mã NPL/SP', 'Đơn vị tính', 'Tổng trị giá', 'Thuế suất XNK']).aggregate({'Tổng số lượng': 'sum'})

e62_df.reset_index(inplace=True)
a12_df.reset_index(inplace=True)


a12_df.rename(columns={
    'Mã NPL/SP': 'nvl_code',
    'Số TK': 'import_cd',
    'Tên hàng': 'nvl_name',
    'Đơn vị tính': 'nvl_unit',
    'Tổng số lượng': 'import_qty'
}, inplace=True)
e62_df.rename(columns={
    'Mã NPL/SP': 'tp_code',
    'Tổng số lượng': 'tp_qty',
    'Số TK': 'export_cd',
}, inplace=True)


export_df = pd.merge(e62_df, bom_df, how='left', on='tp_code')

export_df['export_qty'] = export_df['tp_qty'] * export_df['bom']

df = pd.merge(a12_df, export_df, how="outer", on=['nvl_code', 'Ngày ĐK'], sort=False, indicator=False, validate=None)

df = df.loc[df['nvl_code'].isin(a12_df['nvl_code'].unique())]
df.sort_values(['nvl_code', 'Ngày ĐK'], inplace=True)

df.reset_index(inplace=True)
del df['index']

import_cd = []
import_qt = []
export_cd = []
export_qty = []

current_import_cd = ''
current_export_cd = ''
current_code = None
current_date = ''
current_tp = None

current_import_cds = []
current_export_cds = []

code_to_deletes = []
row_to_deletes = []

df.sort_values(['nvl_code', 'Ngày ĐK'], inplace=True)
df.reset_index(inplace=True)
del df['index']


for count, row in enumerate(zip(
        df['nvl_code'], df['import_cd'], df['export_cd'], 
        df['Ngày ĐK'], df['import_qty'], df['export_qty'], df['tp_code']
    )):
    if current_code:
        if current_code == row[0]:
            current_import_cds.append(row[1])
            current_export_cds.append(row[2])
        else:
            if not [x for x in current_import_cds if not pd.isna(x)] or not [x for x in current_export_cds if not pd.isna(x)]:
                code_to_deletes.append(current_code)
            
            k = 0
            for i in range(len(current_export_cds)):
                if pd.isna(current_import_cds[i]) or not current_import_cds[i]:
                    k += 1
                else:
                    break
            if k > 0:
                row_to_deletes += range(count-len(current_export_cds), count-len(current_export_cds)+k)

            current_import_cds = []
            current_export_cds = []
            current_import_cds.append(row[1])
            current_export_cds.append(row[2])


        if current_import_cd == row[1] and current_import_cd:
            if current_date == row[3] and current_code == row[0]:
                import_cd.append(None)
                import_qt.append(0)
            else:
                import_cd.append(row[1])
                import_qt.append(row[4])
        else:
            import_cd.append(row[1])
            import_qt.append(row[4])

        if current_export_cd == row[2] and current_export_cd:
            
            if current_date == row[3] and current_code == row[0] and current_tp == row[6]:
                export_cd.append(None)
                export_qty.append(0)
            else:
                export_cd.append(row[2])
                export_qty.append(row[5])
        else:
            export_cd.append(row[2])
            export_qty.append(row[5])
    else:
        import_cd.append(row[1])
        import_qt.append(row[4])
        export_cd.append(row[2])
        export_qty.append(row[5])

        current_import_cds.append(row[1])
        current_export_cds.append(row[2])

    current_code = row[0]
    current_date = row[3]
    current_import_cd = row[1]
    current_export_cd = row[2]
    current_tp = row[6]


df['import_cd'] = import_cd
df['export_cd'] = export_cd
df['import_qty'] = import_qt
df['export_qty'] = export_qty

df = df.drop(row_to_deletes)
df = df.loc[~df['nvl_code'].isin(code_to_deletes)]



df['export_date'] = df['Ngày ĐK']
df.rename(columns={'Ngày ĐK': 'import_date'}, inplace=True)

df.sort_values(['nvl_code', 'export_date'], inplace=True)
df.reset_index(inplace=True)
del df['index']

export_null = 0
import_null = 0

nvl_names = []
nvl_units = []

import_cds = []
import_dates = []
import_qty = []

tax_totals = []
tax_rates = []

export_cds = []
export_dates = []
export_qty = []
tps = []
tp_units = []
tp_names = []
bom_values = []

to_delete = []

current_code = None

for count, row in df.iterrows():
    if not current_code:
        if not pd.isna(row['import_cd']):
            import_cds.append(row['import_cd'])
            nvl_names.append(row['nvl_name_x'])
            nvl_units.append(row['nvl_unit_x'])
            import_dates.append(row['import_date'])
            import_qty.append(row['import_qty'])
            tax_totals.append(row['Tổng trị giá'])
            tax_rates.append(row['Thuế suất XNK'])
        else:
            import_null += 1

        if not pd.isna(row['export_cd']):
            export_cds.append(row['export_cd'])
            export_dates.append(row['export_date'])
            export_qty.append(row['export_qty'])
            tps.append(row['tp_code'])
            tp_units.append(row['tp_unit'])
            tp_names.append(row['tp_name'])
            bom_values.append(row['bom'])

        else:
            export_null += 1

    else:
        if current_code == row['nvl_code']:
            if not pd.isna(row['import_cd']):
                import_cds.append(row['import_cd'])
                nvl_names.append(row['nvl_name_x'])
                nvl_units.append(row['nvl_unit_x'])
                import_dates.append(row['import_date'])
                import_qty.append(row['import_qty'])
                tax_totals.append(row['Tổng trị giá'])
                tax_rates.append(row['Thuế suất XNK'])
            else:
                import_null += 1

            if not pd.isna(row['export_cd']):
                export_cds.append(row['export_cd'])
                export_dates.append(row['export_date'])
                export_qty.append(row['export_qty'])
                tps.append(row['tp_code'])
                tp_units.append(row['tp_unit'])
                tp_names.append(row['tp_name'])
                bom_values.append(row['bom'])
            else:
                export_null += 1

            if count == df.shape[0] - 1:
                j = min([import_null, export_null])
                to_delete += range(count-j, count)
                import_cds += [None] * import_null
                nvl_names += [None] * import_null
                nvl_units += [None] * import_null
                import_dates += [None] * import_null
                import_qty += [None] * import_null
                tax_totals += [None] * import_null
                tax_rates += [None] * import_null

                export_cds += [None] * export_null
                export_dates += [None] * export_null
                export_qty += [None] * export_null
                tps += [None] * export_null
                tp_units += [None] * export_null
                tp_names += [None] * export_null
                bom_values += [None] * export_null
    
        else:
            j = min([import_null, export_null])
            to_delete += range(count-j, count)

            import_cds += [None] * import_null
            nvl_names += [None] * import_null
            nvl_units += [None] * import_null
            import_dates += [None] * import_null
            import_qty += [None] * import_null
            tax_totals += [None] * import_null
            tax_rates += [None] * import_null

            export_cds += [None] * export_null
            export_dates += [None] * export_null
            export_qty += [None] * export_null
            tps += [None] * export_null
            tp_units += [None] * export_null
            tp_names += [None] * export_null
            bom_values += [None] * export_null
            
            import_null = 0
            export_null = 0

            if pd.isna(row['import_cd']):
                import_null = 1
            else:
                import_cds.append(row['import_cd'])
                nvl_names.append(row['nvl_name_x'])
                nvl_units.append(row['nvl_unit_x'])
                import_dates.append(row['import_date'])
                import_qty.append(row['import_qty'])
                tax_totals.append(row['Tổng trị giá'])
                tax_rates.append(row['Thuế suất XNK'])

            if pd.isna(row['export_cd']):
                export_null = 1
            else:
                export_cds.append(row['export_cd'])
                export_dates.append(row['export_date'])
                export_qty.append(row['export_qty'])
                tps.append(row['tp_code'])
                tp_units.append(row['tp_unit'])
                tp_names.append(row['tp_name'])
                bom_values.append(row['bom'])

    current_code = row['nvl_code']

export_cds.append(None)
export_dates.append(None)
export_qty.append(None)
tps.append(None)
tp_units.append(None)
tp_names.append(None)
bom_values.append(None)

df['import_cd'] = import_cds
df['export_cd'] = export_cds
df['nvl_name_x'] = nvl_names
df['nvl_unit_x'] = nvl_units
df['import_date'] = import_dates
df['import_qty'] = import_qty
df['Tổng trị giá'] = tax_totals
df['Thuế suất XNK'] = tax_rates
df['export_date'] = export_dates
df['export_qty'] = export_qty
df['tp_code'] = tps
df['tp_unit'] = tp_units
df['tp_name'] = tp_names
df['bom'] = bom_values

df = df.drop(to_delete)
df.sort_values(['nvl_code', 'export_date'], inplace=True)
df.reset_index(inplace=True)
del df['index']



current_code = None
current_import = []
current_export = []
current_import_cd = []
current_nvl_names = []
current_nvl_units = []
current_import_cds = []
current_import_dates = []
current_import_qty = []
current_tax_totals = []
current_tax_rates = []


nvl_names = []
nvl_units = []
import_cds = []
import_dates = []
import_qty = []
tax_totals = []
tax_rates = []


export_cds = []
export_dates = []
export_qty = []
tps = []
tp_units = []
tp_names = []
bom_values = []

to_delete = []

for count, row in df.iterrows():
    if not current_code:
        current_import_cd.append(row['import_cd'])
        current_import.append(row['import_qty'])
        current_export.append(row['export_qty'])
        current_nvl_names.append(row['nvl_name_x'])
        current_nvl_units.append(row['nvl_unit_x'])
        current_import_dates.append(row['import_date'])
        current_import_qty.append(row['import_qty'])
        current_tax_totals.append(row['Tổng trị giá'])
        current_tax_rates.append(row['Thuế suất XNK'])

    else:
        if current_code == row['nvl_code']:
            current_import_cd.append(row['import_cd'])
            current_import.append(row['import_qty'])
            current_export.append(row['export_qty'])
            current_nvl_names.append(row['nvl_name_x'])
            current_nvl_units.append(row['nvl_unit_x'])
            current_import_dates.append(row['import_date'])
            current_import_qty.append(row['import_qty'])
            current_tax_totals.append(row['Tổng trị giá'])
            current_tax_rates.append(row['Thuế suất XNK'])

            if count == df.shape[0] - 1:
                k = 0
                im = 0
                ex = 0
                dice = [0]
                
                length = len(current_import)
                im = current_import.pop(0)

                for i in range(length):
                    ex += current_export[i]
                    if ex < im:
                        pass
                    else:
                        
                        try:
                            im += current_import.pop(0)
                            dice.append(i)
                        except:
                            dice.append(i)
                        
                # --------------------------
                clean_import_cd = [x for x in current_import_cd if x]
                clean_nvl_names = [x for x in current_nvl_names if x]
                clean_nvl_units = [x for x in current_nvl_units if x]
                clean_import_dates = [x for x in current_import_dates if x]
                clean_import_qty = [x for x in current_import_qty if x]
                clean_tax_totals = [x for x in current_tax_totals if x]
                clean_tax_rates = [x for x in current_tax_rates if x]

                temp_cd = []
                temp_nvl_names = []
                temp_nvl_units = []
                temp_import_dates = []
                temp_import_qty = []
                temp_tax_totals = []
                temp_tax_rates = []


                for i in range(len(dice)):
                    try:
                        temp_cd.append(clean_import_cd.pop(0))
                        temp_nvl_names.append(current_nvl_names.pop(0))
                        temp_nvl_units.append(current_nvl_units.pop(0))
                        temp_import_dates.append(current_import_dates.pop(0))
                        temp_import_qty.append(current_import_qty.pop(0))
                        temp_tax_totals.append(current_tax_totals.pop(0))
                        temp_tax_rates.append(current_tax_rates.pop(0))
                    except IndexError:
                        break

                    if i < len(dice) - 1:
                        temp_cd += [None] * (dice[i+1] - dice[i] - 1)
                        temp_nvl_names += [None] * (dice[i+1] - dice[i] - 1)
                        temp_nvl_units += [None] * (dice[i+1] - dice[i] - 1)
                        temp_import_dates += [None] * (dice[i+1] - dice[i] - 1)
                        temp_import_qty += [None] * (dice[i+1] - dice[i] - 1)
                        temp_tax_totals += [None] * (dice[i+1] - dice[i] - 1)
                        temp_tax_rates += [None] * (dice[i+1] - dice[i] - 1)


                if len(temp_cd) < len(current_export):
                    temp_cd.append(None)
                    temp_nvl_names.append(None)
                    temp_nvl_units.append(None)
                    temp_import_dates.append(None)
                    temp_import_qty.append(None)
                    temp_tax_totals.append(None)
                    temp_tax_rates.append(None)
                
                to_delete += range(count-(len(current_export) - len(temp_cd)), count)
                import_cds += temp_cd
                nvl_names += temp_nvl_names
                nvl_units += temp_nvl_units
                import_dates += temp_import_dates
                import_qty += temp_import_qty
                tax_totals += temp_tax_totals
                tax_rates += temp_tax_rates


        else:
            # GET DICE FOR DEVIDE LIST
            k = 0
            im = 0
            ex = 0
            dice = [0]
            
            length = len(current_import)
            im = current_import.pop(0)

            for i in range(length):
                

                ex += current_export[i]
                if ex <= im:
                    pass
                else:
                    try:
                        im += current_import.pop(0)
                        dice.append(i)
                    except:
                        dice.append(i)
            # --------------------------
            clean_import_cd = [x for x in current_import_cd if x]
            clean_nvl_names = [x for x in current_nvl_names if x]
            clean_nvl_units = [x for x in current_nvl_units if x]
            clean_import_dates = [x for x in current_import_dates if x]
            clean_import_qty = [x for x in current_import_qty if x]
            clean_tax_totals = [x for x in current_tax_totals if x]
            clean_tax_rates = [x for x in current_tax_rates if x]

            temp_cd = []
            temp_nvl_names = []
            temp_nvl_units = []
            temp_import_dates = []
            temp_import_qty = []
            temp_tax_totals = []
            temp_tax_rates = []


            for i in range(len(dice)):
                try:
                    temp_cd.append(clean_import_cd.pop(0))
                    temp_nvl_names.append(current_nvl_names.pop(0))
                    temp_nvl_units.append(current_nvl_units.pop(0))
                    temp_import_dates.append(current_import_dates.pop(0))
                    temp_import_qty.append(current_import_qty.pop(0))
                    temp_tax_totals.append(current_tax_totals.pop(0))
                    temp_tax_rates.append(current_tax_rates.pop(0))
                except IndexError:
                    break

                if i < len(dice) - 1:
                    temp_cd += [None] * (dice[i+1] - dice[i] - 1)
                    temp_nvl_names += [None] * (dice[i+1] - dice[i] - 1)
                    temp_nvl_units += [None] * (dice[i+1] - dice[i] - 1)
                    temp_import_dates += [None] * (dice[i+1] - dice[i] - 1)
                    temp_import_qty += [None] * (dice[i+1] - dice[i] - 1)
                    temp_tax_totals += [None] * (dice[i+1] - dice[i] - 1)
                    temp_tax_rates += [None] * (dice[i+1] - dice[i] - 1)


            if len(temp_cd) < len(current_export):
                temp_cd.append(None)
                temp_nvl_names.append(None)
                temp_nvl_units.append(None)
                temp_import_dates.append(None)
                temp_import_qty.append(None)
                temp_tax_totals.append(None)
                temp_tax_rates.append(None)
            
            to_delete += range(count-(len(current_export) - len(temp_cd)), count)
            import_cds += temp_cd
            nvl_names += temp_nvl_names
            nvl_units += temp_nvl_units
            import_dates += temp_import_dates
            import_qty += temp_import_qty
            tax_totals += temp_tax_totals
            tax_rates += temp_tax_rates


            current_import = []
            current_export = []
            current_import_cd = []
            current_nvl_names = []
            current_nvl_units = []
            current_import_cds = []
            current_import_dates = []
            current_import_qty = []
            current_tax_totals = []
            current_tax_rates = []


            current_import_cd.append(row['import_cd'])
            current_import.append(row['import_qty'])
            current_export.append(row['export_qty'])
            current_nvl_names.append(row['nvl_name_x'])
            current_nvl_units.append(row['nvl_unit_x'])
            current_import_dates.append(row['import_date'])
            current_import_qty.append(row['import_qty'])
            current_tax_totals.append(row['Tổng trị giá'])
            current_tax_rates.append(row['Thuế suất XNK'])

    current_code = row['nvl_code']

df = df.drop(to_delete)


df['import_cd'] = import_cds
df['nvl_name_x'] = nvl_names
df['nvl_unit_x'] = nvl_units
df['import_date'] = import_dates
df['import_qty'] = import_qty
df['Tổng trị giá'] = tax_totals
df['Thuế suất XNK'] = tax_rates

df.sort_values(['nvl_code', 'export_date'], inplace=True)
df.reset_index(inplace=True)
del df['index']


df["begin"] = 0
df['import_qty'].fillna(0, inplace=True)
df['export_qty'].fillna(0, inplace=True)
df['temp_import_quantity'] = df.groupby('nvl_code')['import_qty'].cumsum()
df['temp_export_quantity'] = df.groupby('nvl_code')['export_qty'].cumsum()
df['end'] = df['begin'] + df['temp_import_quantity'] - df['temp_export_quantity']
df['begin'] = df['end'] + df['export_qty'] - df['import_qty']

df = df.drop(df[(df['begin'] < 0) & (df['end'] < 0) & (df['import_qty'] == 0)].index)



def revise_export_qty(row):
    if row['end'] < 0:
        if row['begin'] + row['import_qty'] > 0:
            return row['begin'] + row['import_qty']
        else:
            return 0
    return row['export_qty']

def revise_import_qty(row, column):
    if row[column] == 0:
        return None
    return row[column]


df['export_qty'] = df.apply(lambda row: revise_export_qty(row), axis=1)

df['begin'] = 0
df['import_qty'].fillna(0, inplace=True)
df['export_qty'].fillna(0, inplace=True)
df['temp_import_quantity'] = df.groupby('nvl_code')['import_qty'].cumsum()
df['temp_export_quantity'] = df.groupby('nvl_code')['export_qty'].cumsum()
df['end'] = df['begin'] + df['temp_import_quantity'] - df['temp_export_quantity']
df['begin'] = df['end'] + df['export_qty'] - df['import_qty']

df['Tổng trị giá'].fillna(0, inplace=True)
df['Thuế suất XNK'].fillna(0, inplace=True)

df['Tổng trị giá'] =  df['Tổng trị giá'].astype(float)
df['Thuế suất XNK'] =  df['Thuế suất XNK'].astype(float)

df['tax'] = df['Tổng trị giá'] * df['Thuế suất XNK'] / 100

df['import_qty'] = df.apply(lambda row: revise_import_qty(row, column='import_qty'), axis=1)
df['Tổng trị giá'] = df.apply(lambda row: revise_import_qty(row, column='Tổng trị giá'), axis=1)
df['Thuế suất XNK'] = df.apply(lambda row: revise_import_qty(row, column='Thuế suất XNK'), axis=1)
df['tax'] = df.apply(lambda row: revise_import_qty(row, column='tax'), axis=1)

df = df[['import_date', 'import_cd', 'nvl_name_x', 'import_qty', 'nvl_unit_x', 'Tổng trị giá', 'Thuế suất XNK', 'tax' ,'export_date', 'export_cd', 'tp_name', 'export_qty', 'tp_unit', 'nvl_code', 'tp_code', 'bom']]

df.to_excel('../output/refund.xlsx')
df.ffill(inplace=True)

df = df.groupby(
    ['nvl_code', 'import_date', 'import_cd', 'nvl_name_x', 'nvl_unit_x', 'import_qty', 'Tổng trị giá', 'Thuế suất XNK', 'tax' ,'export_date', 'export_cd', 'tp_name', 'export_qty', 'tp_unit', 'tp_code'],
    sort=False
).aggregate({'bom': 'sum'})

In [50]:
len(['2099-04357-00', '3101-07249-00', '3101-07328-00', '3101-07328-01', '3101-07951-00', '3101-07952-00', '3101-07953-00', '3101-07954-00', '3101-08261-00', '3101-08298-00', '3101-08299-00', '3101-08300-00', '3101-08301-00', '3101-08303-00', '3101-08368-00', '3101-08369-00', '3103-05748-00', '3103-08038-01', '3103-08038-02', '3103-08103-00', '3103-08104-00', '3103-08105-00', '3103-08106-00', '3103-08107-00', '3103-08108-00', '3103-08148-03', '3104-00856-01', '3104-00857-01', '3104-00858-01', '3104-00859-01', '3104-00860-01', '3104-00862-01', '3104-00864-01', '3104-00873-00', '3104-00876-00', '3105-00049-00', '3105-00076-00', '3105-00109-00', '3105-00112-00', '3207-01495-01', '3207-01525-07', '3900-09884-04', '3900-11205-00', '8000-03105-00', '8000-03177-00', '8000-03233-00', '8900-00013-02', '9402-07017-72', '9402-07017-73', '9403-03139-40', '9403-03139-43', '9403-12464-22', '9407-03061-16', '9407-03064-25', '9407-03064-26', '9407-04000-01', '9407-04000-04', '9407-04000-07', '9407-04388-86', '9407-04397-55', '9407-04397-56', '9407-04397-57', '9407-14000-08', '9407-14000-09', '9407-15898-64', '9407-15898-65', '9407-15900-10', '9408-10010-89', '9408-10010-90', '9408-10010-91', '9408-10010-92', '9408-10010-93', '9408-10010-94', '9408-10010-95', '9408-10010-96', '9408-10010-97', '9408-10011-01', '9408-10011-02'])

78

In [14]:
a12_df.loc[a12_df.duplicated(subset=['nvl_code', 'Ngày ĐK'])].to_excel('../output/dup.xlsx')